# Google Photos Date Update

## Background

This script is created for a specific use case to update whatsapp photos/videos uploaded to google photos.

Whenever we turn on google photos to automatically upload photos on to an existing folder, it will upload photos based on the system created date and time. On the other hand, whenever we factory reset our phone or change a new phone, whatsapp will download all the backup images and videos onto the new phone again, albeit with the downloaded date and time.

The problem exists when we get a new phone and turn on google photos automatic upload on a newly downloaded whatsapp folder of past images and videos. Google photos will upload all these images and videos and update it as the downloaded date although the actual photo shared date can be glean from the images and video file name.

Take for example the following whatsapp image:
- As of 1 January 2021, google photos was not turn on to automatically back up whatsapp images
- First received via whatsapp on 1 January 2021. The image file name would be saved as 'IMG-20210101-WA0001.jpg'
- Image was backed up by whatsapp to a cloud storage
- On 11 Aug 2022, new phone was purchased. Whatsapp was restored on new phone
- Image 'IMG-20210101-WA0001.jpg' was restored with creation date as 11 Aug 2022 <-- WRONG DATE
- Google photos was turn on to automatic back up whatsapp images
- Google photos upload 'IMG-20210101-WA0001.jpg' assuming it was taken on 11 Aug 2022

This script will update google photos of the correct date that the image or video is received based on the file name. For the above example, the correct date would be 1 January 2021

## Challenges

- Attempt was initially made to make use of google api to update the date of the backed up photos. However, this attempt was unsuccessful as google api does not allow updating of metadata on photos not originally uploaded via the API. Source: https://developers.google.com/photos/library/reference/rest/v1/mediaItems/patch"

- Attempt was made to use selenium to automatically open a browser instance, navigate to google photos and log in. However, this was not successful as google is strict on preventing user from loggin in using automated software such as selenium.
    - Note that this can be overcome using `undetected_chromedriver` package Source: https://github.com/ultrafunkamsterdam/undetected-chromedriver
    - As such, we arrived at the following script which require users to create a new instance of microsoft edge browser and then attaching selenium to the browswer in order to control it automatically. We are able to overcome the google authentication problem this way

## Step 1

Run the following code in command prompt
`msedge.exe --remote-debugging-port=9222 --user-data-dir="..\EdgeProfile"`

## Step 2

Run the following code in sequence

In [6]:
import undetected_chromedriver as uc
import random,time,os,sys,re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.edge.options import Options

In [7]:
#Attach selenium to edge instance
edge_options = Options()
edge_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
#Change chrome driver path accordingly
edge_driver = "..\\..\\Google Photos Date update\\edgedriver_win64\\msedgedriver.exe"
driver = webdriver.Edge(edge_driver, options=edge_options)
print(driver.title)

C:\Users\kyuno\AppData\Local\Temp\ipykernel_18524\113437125.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(edge_driver, options=edge_options)


New tab


In [8]:
#navigate to google photos
driver.get("https://photos.google.com/")

## Step 3

- Login to google photos manually using the web browser
- Select the whatsapp image/video on google photos where the date is updated wrongly

## Step 4

In [9]:
#click on the more information button
driver.find_element(By.CSS_SELECTOR,'button[aria-label="Open info"]').send_keys(Keys.RETURN)

In [14]:
#Perform a loop of 1000 cycle to update the photos date
for x in range(1000):
    #find the filename
    #pic_name = driver.find_element(By.CSS_SELECTOR,'div[aria-label*="Filename: IMG"]')
    pic_name = driver.find_element(By.XPATH,'(//div[@aria-label = contains(.,"Filename: IMG")])')
    print(f'\nThis is the {x+1} photo')
    print('1) '+ driver.current_url)    
    time.sleep(0.2)
    
    #for every 5 photos, open and close the edit button. This helps to reduces the memory load. Otherwise, after updating around 50 images, the browser will hang due to memory overload
    if x>0 and x%5 == 0:
        time.sleep(1)
        driver.find_element(By.XPATH,"html").send_keys("e")
        time.sleep(2)
        driver.find_element(By.XPATH,"html").send_keys("e")
        time.sleep(2)
        print('**open and close edit side panel**')
    
    try:
        #sort out the date data
        match = re.search("-(\d{4})(\d{2})(\d{2})",pic_name.text)
        print('2) '+ match.group(1),match.group(2),match.group(3))
        time.sleep(0.2)
        
        #click on the date time edit button by finding parent that does not have 'display:none'
        #Note to self: to provide dual conditions such as class name and then the 2nd instance of that class name, bracket the first condition (eg. [the class name]), then use square brackets again to input the second condition (eg.[2] to select the second instance) 
        driver.find_element(By.XPATH,"((//c-wiz[@class='WUbige'])[not(contains(@style,'display: none;'))])/*/*/*/*/div[@class='ffq9nc Kd04rd']").send_keys(Keys.RETURN)
        print('3) pressed edit button')
        time.sleep(1)
        
        #input the year, month, day with the data extract from the file name
        driver.find_element(By.CSS_SELECTOR,'input[aria-label="Year"]').send_keys(match.group(1))
        driver.find_element(By.CSS_SELECTOR,'input[aria-label="Month"]').send_keys(match.group(2))
        driver.find_element(By.CSS_SELECTOR,'input[aria-label="Day"]').send_keys(match.group(3))
        print('4) keyed in correct date')
        
        #click save
        driver.find_element(By.XPATH,"(//button[@class='VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ nCP5yc AjY5Oe LQeN7 kHssdc HvOprf'])[2]").send_keys(Keys.RETURN)
        print('5) pressed saved button')
        time.sleep(1)
        
    except:
        print('something went wrong!')
        pass

    
    #click next to go to next photo
    driver.find_element(By.XPATH,"html").send_keys(Keys.RIGHT) #this is more stable
    if x>0 and x%5 == 0:
        time.sleep(5)
    else:
        time.sleep(2)
    


This is the 1 photo
1) https://photos.google.com/search/11%20Aug%202022/photo/AF1QipOTqLd7-p2gor4ZlkrmXSTRCEX8eNXRB3DjOXOX
2) 2021 01 01
3) pressed edit button
4) keyed in correct date
5) pressed saved button


KeyboardInterrupt: 